# Practical Work: Fire Detection

In [1]:
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
# from torchsummary import summary

import matplotlib.pyplot as plt

from datasets import WildfirePredictionDataset

In [ ]:
# Set random seed for reproducability and dataset splitting
torch.manual_seed(42)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

## Exploring Data

In [ ]:
# Define image transformation
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load Wildfire Prediction Dataset dataset
dataset = WildfirePredictionDataset(split="valid", transform=transforms)
print(len(dataset))

In [ ]:
sample_idx = torch.randint(len(dataset), size=(1,)).item()
img, label = dataset[sample_idx]
print(img.shape)

In [ ]:
# Plot images and labels
figure = plt.figure(figsize=(9, 9))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(dataset), size=(1,)).item()
    img, label = dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(f"{label} | {labels_dict[label]}")
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()